In [1]:
With_OneAPI = 1 # Make sure to change environment
Model = "Mental Health Chatbot"

In [2]:
import os
if With_OneAPI == 1:
    os.environ['TF_ENABLE_ONEDNN_OPTS']='1'
    os.environ['TF_ENABLE_MKL_NATIVE_FORMAT']='1'
    os.environ['KMP_BLOCKTIME']='1'
    os.environ['KMP_SETTINGS']='1'
    os.environ['OMP_NUM_THREADS']='6'
    os.environ['KMP_AFFINITY']='granularity=fine,verbose,compact,1,0'
    import tensorflow as tf
    tf.config.threading.set_inter_op_parallelism_threads(2) 
    tf.config.threading.set_intra_op_parallelism_threads(6)
    tf.config.set_soft_device_placement(False)
else:
    import tensorflow as tf
    

In [3]:
import numpy as np
import pandas as pd
from sklearnex import patch_sklearn
patch_sklearn() 
import modin.pandas as pd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import json
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

with open(r'./dataset/intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])

2023-07-01 15:25:49,042	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [5]:
df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,gibberish,[jhjbdsjcxn],"[I'm sorry, I couldn't understand your message..."
79,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [6]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

In [7]:
df = pd.DataFrame.from_dict(dic)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [8]:
df

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
228,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
229,gibberish,jhjbdsjcxn,"[I'm sorry, I couldn't understand your message..."
230,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
231,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [9]:
df.to_csv('./dataset/dataframe_chatbot.csv', index=True)

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))


number of unique words =  304
X shape =  (233, 18)
y shape =  (233,)
num of classes =  81


In [11]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           30500     
                                                                 
 lstm (LSTM)                 (None, 18, 32)            17024     
                                                                 
 layer_normalization (LayerN  (None, 18, 32)           64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_1 (Laye  (None, 18, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

In [12]:
import time
start = time.time()
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=30)
end = time.time()

Epoch 1/30
24/24 [==============================] - 8s 14ms/step - loss: 4.9000 - accuracy: 0.0343
Epoch 2/30
24/24 [==============================] - 0s 15ms/step - loss: 3.7195 - accuracy: 0.1245
Epoch 3/30
24/24 [==============================] - 0s 14ms/step - loss: 3.1352 - accuracy: 0.2704
Epoch 4/30
24/24 [==============================] - 0s 14ms/step - loss: 2.6165 - accuracy: 0.3605
Epoch 5/30
24/24 [==============================] - 0s 14ms/step - loss: 2.1855 - accuracy: 0.4979
Epoch 6/30
24/24 [==============================] - 0s 14ms/step - loss: 1.8033 - accuracy: 0.5622
Epoch 7/30
24/24 [==============================] - 0s 15ms/step - loss: 1.4704 - accuracy: 0.6652
Epoch 8/30
24/24 [==============================] - 0s 14ms/step - loss: 1.1654 - accuracy: 0.7382
Epoch 9/30
24/24 [==============================] - 0s 14ms/step - loss: 0.8592 - accuracy: 0.8155
Epoch 10/30
24/24 [==============================] - 0s 14ms/step - loss: 0.7773 - accuracy: 0.8326
Epoch 11/

In [13]:
import re
import random

def model_responce(query, model):
        text = []
        txt = re.sub('[^a-zA-Z\']', ' ', query)
        txt = txt.lower()
        txt = txt.split()
        txt = " ".join(txt)
        text.append(txt)
        x_test = tokenizer.texts_to_sequences(text)
        if (len(query.split(' ')) > 1):
                x_test = np.array(x_test).squeeze()
        else:
                x_test = np.reshape(x_test, -1)
        try:
                x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
        except Exception as e:
                return "Error 404: Word not found in my dictionary"
        y_pred = model.predict(x_test)
        y_pred = y_pred.argmax()
        tag = lbl_enc.inverse_transform([y_pred])[0]
        responses = df[df['tag'] == tag]['responses'].values[0]

        print("you: {}".format(query))
        print("model: {}".format(random.choice(responses)))
        return random.choice(responses)

In [14]:
model_responce('Hi! How are you?', model)

1/1 [==============================] - 2s 2s/step
you: Hi! How are you?
model: I was created by Phoenix 13.


'I was created by Phoenix 13.'

In [15]:
model_responce('How can I maintain social connections? What if I feel lonely?', model)

1/1 [==============================] - 0s 18ms/step
you: How can I maintain social connections? What if I feel lonely?
model: A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help. The quality of your social connections matter. Mindlessly scrolling through social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included and connected. If your current strategies don't help you feel con

"A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help. The quality of your social connections matter. Mindlessly scrolling through social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included and connected. If your current strategies don't help you feel connected, problem-solve to see if you can find a solution. Everyone feels lonely at times. Maybe you recently moved to a new ci

In [16]:
model_responce('Well... I\'m feeling sad!', model)

1/1 [==============================] - 0s 21ms/step
you: Well... I'm feeling sad!
model: I was trained on a text dataset using Deep Learning & Natural Language Processing techniques


'The real question is: Who created you?'

In [17]:
model_responce('Maybe I just didn\'t want to be born :)', model)

1/1 [==============================] - 0s 27ms/step
you: Maybe I just didn't want to be born :)
model: Talking about something really helps. If you're not ready to open up then that's ok. Just know that i'm here for you, whenever you need me.


'You can talk to me without fear of judgement.'

In [18]:
model_responce('I want commit suicide', model)

1/1 [==============================] - 0s 23ms/step
you: I want commit suicide
model: I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.


"I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821."

In [19]:
model_responce('I am facing a lot of problems', model)

1/1 [==============================] - 0s 19ms/step
you: I am facing a lot of problems
model: I am sorry to hear that. What is the reason behind this?


'I am sorry to hear that. What is the reason behind this?'

In [20]:
model_responce('i do not think you can help me', model)

1/1 [==============================] - 0s 24ms/step
you: i do not think you can help me
model: It sound like i'm not being very helpful right now.


"It sound like i'm not being very helpful right now."

In [21]:
model_responce('I need help', model)

1/1 [==============================] - 0s 27ms/step
you: I need help
model: Tell me your problem so that i can assist you


'Sure. Tell me how can i assist you'

In [22]:
model_responce('Good morning', model)

1/1 [==============================] - 0s 27ms/step
you: Good morning
model: Good morning. I hope you had a good night's sleep. How are you feeling today? 


"Good morning. I hope you had a good night's sleep. How are you feeling today? "

In [23]:
model_responce('Good evening', model)

1/1 [==============================] - 0s 20ms/step
you: Good evening
model: Good evening. How has your day been?


'Good evening. How has your day been?'

In [24]:
model_responce('Gosadj kjasndkja sadan  kadsjnbkjabdhjk', model)

1/1 [==============================] - 0s 18ms/step
you: Gosadj kjasndkja sadan  kadsjnbkjabdhjk
model: Sorry, I didn't understand you.


"Sorry, I didn't understand you."

In [25]:
model.save('./chatbot_model.h5')

In [26]:
test_list = ['Hi! How are you?', 
             'How can I maintain social connections? What if I feel lonely?', 
             'Well... I\'m feeling sad!', 
             'Maybe I just didn\'t want to be born :)', 
             'I want commit suicide', 
             'I am facing a lot of problems', 
             'i do not think you can help me', 
             'I need help', 
             'Good morning', 
             'Good evening', 
             'Gosadj kjasndkja sadan  kadsjnbkjabdhjk',
             "Goodbye",
             "You hate me",
             "You already told me that",
             "Where are you?",
             "yeah you're right. i deserve a break.",
             "I want to learn about mental health.",
             "Why is mental health important?",
             "How do I know if I'm unwell?",
             "How were you made?"]
len(test_list)

20

In [27]:
import time

total_inference_time = []

for test in test_list:

    start_time = time.time()

    _ = model_responce(test, model)

    end_time = time.time()
    inference_time = end_time - start_time
    total_inference_time.append(inference_time)

1/1 [==============================] - 0s 20ms/step
you: Hi! How are you?
model: I was trained on a text dataset using Deep Learning & Natural Language Processing techniques
1/1 [==============================] - 0s 20ms/step


you: How can I maintain social connections? What if I feel lonely?
model: A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help. The quality of your social connections matter. Mindlessly scrolling through social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included and connected. If your current strategies don't help you feel connected, problem-solve to see if you can find a solut

In [28]:
inference_speed = len(test_list) / sum(total_inference_time)
latency = sum(total_inference_time) / len(test_list)
throughput = len(test_list) / sum(total_inference_time)
training_time = end - start

In [29]:
print(f"Inference Speed: {inference_speed} outputs/second")
print(f"Latency: {latency} seconds")
print(f"Throughput: {throughput} outputs/second")
print(f"Training Time: {training_time} seconds")

Inference Speed: 12.422992065440788 outputs/second
Latency: 0.08049590587615967 seconds
Throughput: 12.422992065440788 outputs/second
Training Time: 15.987110614776611 seconds


In [30]:
benchmarks = ["Inference_Speed", "Latency", "Throughput", "Training_Time"] 
results = {"Inference_Speed": inference_speed, 
           "Latency": latency, 
           "Throughput": throughput,
           "Training_Time": training_time}

In [31]:
# import csv

# csv_file_path = './benchmarking.csv'
# if With_OneAPI == 0:
#     column_index = 3
#     # Read the existing CSV file
#     with open(csv_file_path, 'r') as file:
#         reader = csv.reader(file)
#         data = list(reader)

#     # Add the new data to the appropriate columns
#     for i in benchmarks:
#         new_line = [Model, i, '', '']
#         new_line[column_index] = results[i]
#         data.append(new_line)

#     # Write the modified data back to the CSV file
#     with open(csv_file_path, 'w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerows(data)
# else:
#     column_index = 2
#     import pandas as pd

#     df = pd.read_csv("./benchmarking.csv")

#     for i in benchmarks:
#         df.loc[(df['Model'] == Model) & (df["benchmark_parameter"] == i), "With_ONEAPI"] = results[i]

#     df.to_csv('./benchmarking.csv', index= False)
